# **Análisis y Comparación de Modelos de Machine Learning: Redes Neuronales y Árboles de Decisión**

En este proyecto se busca construir modelos de **Machine Learning** que permitan predecir el estado de aprobación de un préstamo utilizando los atributos de las personas solicitantes. Para ello, se analiza un conjunto de datos que contiene 14 atributos, entre ellos la edad, el ingreso anual, el puntaje crediticio, el propósito del préstamo, entre otros. La etiqueta de clase, loan_status, indica si el préstamo fue aprobado (1) o rechazado (0).

Utilizaremos dos técnicas principales para abordar este problema: **Redes Neuronales** y **Árboles de Decisión**, variando sus configuraciones y modificando hiperparámetros clave. El objetivo es comparar la exactitud (accuracy) de los modelos generados y determinar cuáles ofrecen mejores predicciones.

##### INTEGRANTES:
  1. Marcela Mazo Castro - 1843612
  2. Eyder Santiago Suárez Chávez - 2322714
  3. Erika García Muñoz - 2259395
  4. Juan José Moreno Jaramillo - 2310038

## Preparación de los Datos  
En esta sección, se cargan y preparan los datos para el entrenamiento y evaluación de los modelos.  

In [ ]:
# Importar bibliotecas necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Leer el archivo loan_data.csv
data = pd.read_csv("loan_data.csv")

# Dividir aleatoriamente los datos
train_data, test_data = train_test_split(data, test_size=0.2, random_state=123)

# Normalización y codificación de los datos
numerical_features = ["person_age", "person_income", "person_emp_exp", "loan_amnt", "loan_int_rate", 
                      "loan_percent_income", "cb_person_cred_hist_length", "credit_score"]
categorical_features = ["person_gender", "person_education", "person_home_ownership", "loan_intent", 
                        "previous_loan_defaults_on_file"]

preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numerical_features),
    ('cat', OneHotEncoder(), categorical_features)
])

ModuleNotFoundError: No module named 'sklearn'

## Modelos de Redes Neuronales  
En esta sección, se construyen y evalúan 5 modelos de redes neuronales variando la cantidad de capas ocultas, el número de neuronas por capa y los hiperparámetros principales como solver y activation. La implementación utiliza la clase MLPClassifier de scikit-learn para entrenar las redes neuronales con diferentes configuraciones topológicas y evaluar su desempeño en términos de exactitud.



In [ ]:
# 1. Construcción de redes neuronales
topologies = [
    (50,), (100,), (50, 50), (100, 50), (100, 100)
]

results = []

for topology in topologies:
    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', MLPClassifier(hidden_layer_sizes=topology, random_state=123, max_iter=500))
    ])
    
    # Entrenamiento
    model.fit(train_data.drop(columns="loan_status"), train_data["loan_status"])
    
    # Predicción
    predictions = model.predict(test_data.drop(columns="loan_status"))
    accuracy = accuracy_score(test_data["loan_status"], predictions)
    
    results.append({"Topology": topology, "Accuracy": accuracy})

### Análisis de Resultados  
Los resultados de las diferentes configuraciones topológicas se resumen en una tabla, mostrando la exactitud alcanzada por cada red neuronal.

In [ ]:
# 2. Tabla de resultados
results_df = pd.DataFrame(results)
print(results_df)

# 3. Hiperparámetro adicional
# Modificar un hiperparámetro adicional (como alpha)
alpha_variations = [0.0001, 0.01]

for alpha in alpha_variations:
    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', MLPClassifier(hidden_layer_sizes=(100, 50), random_state=123, max_iter=500, alpha=alpha))
    ])
    
    model.fit(train_data.drop(columns="loan_status"), train_data["loan_status"])
    predictions = model.predict(test_data.drop(columns="loan_status"))
    accuracy = accuracy_score(test_data["loan_status"], predictions)
    
    print(f"Alpha: {alpha}, Accuracy: {accuracy}")

## Modelos de Árboles de Decisión  
Se construirán y evaluarán 10 árboles de decisión variando el hiperparámetro `max_depth` desde 1 hasta 10, utilizando dos criterios diferentes: `gini` y `entropy`.  

In [ ]:

from sklearn.tree import DecisionTreeClassifier

# 1 y 2. Leer datos y dividirlos
data = pd.read_csv("loan_data.csv")
train_data, test_data = train_test_split(data, test_size=0.2, random_state=123)

# 3. Normalización y codificación
# Reutilizar el preprocessor del notebook anterior

# 4. Árboles de decisión con max_depth
depth_results = []

for depth in range(1, 11):
    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', DecisionTreeClassifier(criterion="gini", splitter="best", max_depth=depth, random_state=123))
    ])
    
    model.fit(train_data.drop(columns="loan_status"), train_data["loan_status"])
    predictions = model.predict(test_data.drop(columns="loan_status"))
    accuracy = accuracy_score(test_data["loan_status"], predictions)
    
    depth_results.append({"Max Depth": depth, "Accuracy": accuracy})

depth_results_df = pd.DataFrame(depth_results)
print(depth_results_df)

# Repetir el proceso con criterion="entropy"

   Max Depth  Accuracy
0          1  0.781778
1          2  0.853556
2          3  0.902556
3          4  0.916444
4          5  0.919111
5          6  0.919222
6          7  0.919556
7          8  0.923778
8          9  0.923222
9         10  0.924444


## Conclusiones de los Modelos 